## Presentation Types with Hidden Types
## Also Network Visualization with Louvain Communities

* March 2022 version

    * Uses getDistance to identify `close matches` with side-by-side comparison of soggetti.  With a distance of "1", the soggetti `4, 1, 2, 3`, and `5, 1, 2, 3` will count as the same.  These are reported as "flexed entries" in a separate column.

    * Labels Fuga, PEn, and ID according to time intervals.  
    * If two entries are separated by more than 10 bars (80 offsets), the tool resets to a new pattern
    * Finds time intervals between entries (expressed as offsets, like `8.0, 4.0, 8.0`)
    * Finds melodic intervals between first note of successive entries in each pattern (like `P-5, P-8`)
    * Counts number of entries
    * Provides offset and measure/beat locations
    * Sorts all presentation types by the order in which they appear in the piece
    * Reports voice names of the entries, in order of their appearance
    * Omits singleton soggetti (just one entry of a given motive in isolation)
    
    ALSO
    
    * Finds "hidden" types within a longer Fuga.  That is, if a 5-voice fuga also contains a PEN, it will label both of these as separate presentation type, along with all the relevant data noted above.

In [23]:
import intervals
from intervals import * 
from intervals import main_objs
import intervals.visualizations as viz
import pandas as pd
import re
import altair as alt 
from ipywidgets import interact
from pandas.io.json import json_normalize
from pyvis.network import Network
from IPython.display import display
import requests
import os
import numpy as np
import itertools
from itertools import combinations
import networkx as nx
from community import community_louvain
from copy import deepcopy
MYDIR = ("saved_csv")
CHECK_FOLDER = os.path.isdir(MYDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MYDIR)
    print("created folder : ", MYDIR)

else:
    print(MYDIR, "folder already exists.")

saved_csv folder already exists.


#### The following are special functions used by the classifier.  Don't change them.

## Load one Piece Here

* Note that you can load from CRIM, or put a file in the **Music_Files** folder in the Notebook.

In [24]:
git_prefix = 'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/'

# just add the CRIM Piece ID here
mei_file = 'CRIM_Mass_0005_2.mei'


url = git_prefix + mei_file
# piece = importScore('Music_Files/Senfl_Ave_forCRIM.mei_msg.mei')
piece = importScore(url)
# piece = importScore('Music_Files/CRIM_Mass_0007_4.mei')

print(piece.metadata)

Successfully imported https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0005_2.mei
{'title': 'Missa Ave Maria: Gloria', 'composer': 'Antoine de Févin'}


## Run the Classifier Here

- set the length of the soggetti with `melodic_ngram_length`
- set the maximum difference between similar soggetti with `edit_distance_threshold`
- for chromatic vs diatonic, compound, and directed data in soggetti, see `interval_settings`
- to include all the hidden PENs and IDS (those found within longer Fugas, use `include_hidden_types == True`.  
- for faster (and simpler) listing of points of imitation without hidden forms, use `include_hidden_types == False`



In [25]:
include_hidden_types = False
melodic_ngram_length = 4
edit_distance_threshold = 1
nr = piece.getNoteRest()
dur = piece.getDuration(df=nr)
dur_ng = piece.getNgrams(df=dur, n=melodic_ngram_length)
mel_ng = piece.getMelodicEntries(interval_settings=('d', True, True), n=melodic_ngram_length)
output = classify_entries_as_presentation_types(piece, nr, dur_ng, mel_ng, edit_distance_threshold, include_hidden_types)
# len(output)

#### Below Find Source Code and Explanation of the Method

In [26]:
output

,Composer,Title,First_Offset,Measures_Beats,Melodic_Entry_Intervals,Offsets,Soggetti,Time_Entry_Intervals,Voices,Presentation_Type,Number_Entries,Flexed_Entries
0,Antoine de Févin,Missa Ave Maria: Gloria,1002.0,"[124/2.0, 127/2.0]",[P-8],"[1002.0, 1026.0]","[-2, -2, -2, 2]",[24.0],"[Altus, Bassus]",FUGA,2,False
1,Antoine de Févin,Missa Ave Maria: Gloria,168.0,"[22/1.0, 25/1.0]",[M-9],"[168.0, 192.0]","[1, 1, 2, -2, 1, 1, 1, -2]",[24.0],"[Sup[erius], Tenor]",FUGA,2,True
2,Antoine de Févin,Missa Ave Maria: Gloria,382.0,[48/4.0],"[M-6, P-5]","[382.0, 382.0, 382.0]","[1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1]","[0.0, 0.0]","[Sup[erius], Altus, Tenor]",PEN,3,True
3,Antoine de Févin,Missa Ave Maria: Gloria,300.0,"[38/3.0, 40/1.0, 48/4.0]","[P8, M10, M-6, P-5]","[300.0, 312.0, 382.0, 382.0, 382.0]","[1, 1, 2, 2, 1, 1, 1, 2, 1, 1, 1, 1]","[12.0, 70.0, 0.0, 0.0]","[Bassus, Tenor, Sup[erius], Altus, Tenor]",FUGA,5,True
4,Antoine de Févin,Missa Ave Maria: Gloria,92.0,"[12/3.0, 13/1.0]",[P-5],"[92.0, 96.0]","[1, 1, 4, 1, 1, 1, 5, 1]",[4.0],"[Tenor, Bassus]",FUGA,2,True
5,Antoine de Févin,Missa Ave Maria: Gloria,712.0,"[88/1.0, 91/1.0, 94/1.0, 96/1.0, 99/1.0, 101/1.0]","[P-8, P12, P-5, P-4, P-5]","[712.0, 736.0, 760.0, 776.0, 800.0, 816.0]","[1, 1, 5, 1, 1, 1, 5, 2]","[24.0, 24.0, 16.0, 24.0, 16.0]","[Altus, Bassus, Sup[erius], Altus, Tenor, Bassus]",FUGA,6,True
6,Antoine de Févin,Missa Ave Maria: Gloria,240.0,"[31/1.0, 31/3.0]",[P8],"[240.0, 244.0]","[2, 2, 2, -2, 1, 2, 2, -2]",[4.0],"[Altus, Sup[erius]]",FUGA,2,True
7,Antoine de Févin,Missa Ave Maria: Gloria,582.0,"[72/4.0, 73/2.0]",[P-5],"[582.0, 586.0]","[4, -2, -2, -2]",[4.0],"[Altus, Bassus]",FUGA,2,False
8,Antoine de Févin,Missa Ave Maria: Gloria,0.0,"[1/1.0, 3/3.0]",[P8],"[0.0, 20.0]","[4, 1, 2, 2]",[20.0],"[Altus, Sup[erius]]",FUGA,2,False


### Run Classifier on Several Pieces at Once

Results are combined into a single dataframe

In [27]:


git_prefix = 'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/'

# piece = importScore('Music_Files/CRIM_Mass_0007_4.mei')
piece_list =  ['CRIM_Mass_0005_3.mei',
             'CRIM_Mass_0005_4.mei',
             'CRIM_Mass_0005_5.mei',
             'CRIM_Model_0001.mei',
             'CRIM_Mass_0002_1.mei',
             'CRIM_Mass_0002_2.mei',
             'CRIM_Mass_0002_3.mei',
             'CRIM_Mass_0002_4.mei',
             'CRIM_Mass_0002_5.mei',
             'CRIM_Model_0015.mei',
             'CRIM_Mass_0013_1.mei',
             'CRIM_Mass_0013_2.mei',
             'CRIM_Mass_0013_3.mei',
             'CRIM_Mass_0013_4.mei',
             'CRIM_Mass_0013_5.mei',
             'CRIM_Model_0019.mei',
             'CRIM_Mass_0019_1.mei',
             'CRIM_Mass_0019_2.mei',
             'CRIM_Mass_0019_3.mei',
             'CRIM_Mass_0019_4.mei',
             'CRIM_Mass_0019_5.mei']


In [28]:
include_hidden_types = False
melodic_ngram_length = 4
edit_distance_threshold = 1
final = pd.DataFrame()
for work in piece_list:
    url = git_prefix + work
    piece = importScore(url)   
    nr = piece.getNoteRest()
    dur = piece.getDuration(df=nr)
    dur_ng = piece.getNgrams(df=dur, n=melodic_ngram_length)
    mel_ng = piece.getMelodicEntries(interval_settings=('d', True, True), n=melodic_ngram_length)
    output = classify_entries_as_presentation_types(piece, nr, dur_ng, mel_ng, edit_distance_threshold, include_hidden_types)
    final = final.append(output, ignore_index=True)
final.head()

mei.base: WARNING: Importing <slur> without @startid and @endid is not yet supported.


Successfully imported https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0005_3.mei
Successfully imported https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0005_4.mei
Successfully imported https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0005_5.mei
Successfully imported https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Model_0001.mei
Successfully imported https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0002_1.mei
Successfully imported https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0002_2.mei
Successfully imported https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0002_3.mei
Successfully imported https://raw.githubusercontent.com/CRIM-Project/C

,Composer,Title,First_Offset,Measures_Beats,Melodic_Entry_Intervals,Offsets,Soggetti,Time_Entry_Intervals,Voices,Presentation_Type,Number_Entries,Flexed_Entries
0,Antoine de Févin,Missa Ave Maria: Credo,654.0,"[82/4.0, 83/2.0, 86/2.0, 86/4.0, 89/4.0, 92/2.0]","[M-6, m-3, M-6, M13, P-5]","[654.0, 658.0, 682.0, 686.0, 710.0, 730.0]","[-2, -3, 2, 2, -2, -2, 2, 2]","[4.0, 24.0, 4.0, 24.0, 20.0]","[Sup[erius], Altus, Tenor, Bassus, Sup[erius],...",FUGA,6,True
1,Antoine de Févin,Missa Ave Maria: Credo,190.0,"[24/4.0, 25/2.0]",[P5],"[190.0, 194.0]","[-2, -3, 3, -2]",[4.0],"[Bassus, Tenor]",FUGA,2,False
2,Antoine de Févin,Missa Ave Maria: Credo,876.0,"[109/3.0, 111/3.0, 112/3.0]","[P-8, P-5]","[876.0, 892.0, 900.0]","[2, -2, -2, -2, 1, -2, -2, -2]","[16.0, 8.0]","[Sup[erius], Tenor, Bassus]",FUGA,3,True
3,Antoine de Févin,Missa Ave Maria: Credo,1144.0,"[142/1.0, 144/1.0]",[P-5],"[1144.0, 1160.0]","[1, -2, -2, 2, 1, -2, -3, 2]",[16.0],"[Sup[erius], Altus]",FUGA,2,True
4,Antoine de Févin,Missa Ave Maria: Credo,462.0,"[58/4.0, 59/2.0]",[M-6],"[462.0, 466.0]","[1, 1, 1, -2]",[4.0],"[Sup[erius], Altus]",FUGA,2,False


In [11]:
final["MINT"] = final["Melodic_Entry_Intervals"].apply(joiner)
final["TINT"] = final["Time_Entry_Intervals"].apply(joiner)

final['SOG'] = final['Soggetti'].apply(clean_melody)
final['ALL'] = final["MINT"] + '_' + final["TINT"] + '_' + final['SOG']

final["ALL_INT"] = final["MINT"] + '_' + final["TINT"]
final["ALL_SOG"] = final["MINT"] + '_' + final["SOG"]
final

,Composer,Title,First_Offset,Measures_Beats,Melodic_Entry_Intervals,Offsets,Soggetti,Time_Entry_Intervals,Voices,Presentation_Type,Number_Entries,Flexed_Entries,MINT,TINT,SOG,ALL,ALL_INT,ALL_SOG
0,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,394.0,"[50/2.0, 51/4.0, 53/2.0, 55/2.0, 56/4.0, 59/1....","[P5, P-8, P4, P5, P-8, P8, P-4, P-5]","[394.0, 406.0, 418.0, 434.0, 446.0, 464.0, 486...","[-2, 2, 1, 3]","[12.0, 12.0, 16.0, 12.0, 18.0, 22.0, 4.0, 16.0]","[Altus, Cantus, Bassus, Tenor, Altus, Bassus, ...",FUGA,9,False,P5_P-8_P4_P5_P-8_P8_P-4_P-5,12.0_12.0_16.0_12.0_18.0_22.0_4.0_16.0,-2_2_1_3,P5_P-8_P4_P5_P-8_P8_P-4_P-5_12.0_12.0_16.0_12....,P5_P-8_P4_P5_P-8_P8_P-4_P-5_12.0_12.0_16.0_12....,P5_P-8_P4_P5_P-8_P8_P-4_P-5_-2_2_1_3
1,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,0.0,"[1/1.0, 2/1.0, 6/1.0, 7/1.0, 11/3.0, 14/1.0, 1...","[P-5, P-4, P-5, P8, P5, P-8, P1, P8, P-5, P-4,...","[0.0, 8.0, 40.0, 48.0, 84.0, 104.0, 128.0, 150...","[-3, 3, 2, -2, -3, 2, 2, -2]","[8.0, 32.0, 8.0, 36.0, 20.0, 24.0, 22.0, 16.0,...","[Cantus, Altus, Tenor, Bassus, Altus, Cantus, ...",FUGA,15,True,P-5_P-4_P-5_P8_P5_P-8_P1_P8_P-5_P-4_P-5_P8_P8_...,8.0_32.0_8.0_36.0_20.0_24.0_22.0_16.0_8.0_24.0...,-3_3_2_-2,P-5_P-4_P-5_P8_P5_P-8_P1_P8_P-5_P-4_P-5_P8_P8_...,P-5_P-4_P-5_P8_P5_P-8_P1_P8_P-5_P-4_P-5_P8_P8_...,P-5_P-4_P-5_P8_P5_P-8_P1_P8_P-5_P-4_P-5_P8_P8_...
2,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,298.0,"[38/2.0, 38/4.0, 40/2.0, 42/2.0, 44/1.0, 46/2....","[P-5, P8, P-12, P8, P8, P-11]","[298.0, 302.0, 314.0, 330.0, 344.0, 362.0, 382.0]","[1, 1, -3, 3]","[4.0, 12.0, 16.0, 14.0, 18.0, 20.0]","[Altus, Tenor, Cantus, Bassus, Altus, Cantus, ...",FUGA,7,False,P-5_P8_P-12_P8_P8_P-11,4.0_12.0_16.0_14.0_18.0_20.0,1_1_-3_3,P-5_P8_P-12_P8_P8_P-11_4.0_12.0_16.0_14.0_18.0...,P-5_P8_P-12_P8_P8_P-11_4.0_12.0_16.0_14.0_18.0...,P-5_P8_P-12_P8_P8_P-11_1_1_-3_3
3,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Kyrie,458.0,"[56/2.0, 58/4.0, 61/3.0, 63/2.0]","[P-8, P-5, P12]","[458.0, 478.0, 500.0, 514.0]","[-2, 2, 1, 3]","[20.0, 22.0, 14.0]","[Cantus, Tenor, Bassus, Cantus]",FUGA,4,False,P-8_P-5_P12,20.0_22.0_14.0,-2_2_1_3,P-8_P-5_P12_20.0_22.0_14.0_-2_2_1_3,P-8_P-5_P12_20.0_22.0_14.0,P-8_P-5_P12_-2_2_1_3
4,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Kyrie,510.0,"[62/4.0, 64/2.0]",[P5],"[510.0, 522.0]","[-2, 2, 2, 2]",[12.0],"[Tenor, Altus]",FUGA,2,False,P5,12.0,-2_2_2_2,P5_12.0_-2_2_2_2,P5_12.0,P5_-2_2_2_2
5,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Kyrie,0.0,"[1/1.0, 2/1.0, 6/1.0, 7/1.0]","[P-5, P-4, P-5]","[0.0, 8.0, 40.0, 48.0]","[-3, 3, 2, -2, -3, 2, 2, -2]","[8.0, 32.0, 8.0]","[Cantus, Altus, Tenor, Bassus]",ID,4,True,P-5_P-4_P-5,8.0_32.0_8.0,-3_3_2_-2,P-5_P-4_P-5_8.0_32.0_8.0_-3_3_2_-2,P-5_P-4_P-5_8.0_32.0_8.0,P-5_P-4_P-5_-3_3_2_-2
6,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Kyrie,368.0,"[45/1.0, 45/3.0, 47/3.0, 49/3.0, 51/4.0, 52/2....","[P-5, P8, P-8, P8, P-5, P-8]","[368.0, 372.0, 388.0, 404.0, 422.0, 426.0, 436.0]","[1, 1, -3, 3, 1, 1, -3, 2]","[4.0, 16.0, 16.0, 18.0, 4.0, 10.0]","[Altus, Tenor, Cantus, Bassus, Altus, Tenor, B...",FUGA,7,True,P-5_P8_P-8_P8_P-5_P-8,4.0_16.0_16.0_18.0_4.0_10.0,1_1_-3_3,P-5_P8_P-8_P8_P-5_P-8_4.0_16.0_16.0_18.0_4.0_1...,P-5_P8_P-8_P8_P-5_P-8_4.0_16.0_16.0_18.0_4.0_10.0,P-5_P8_P-8_P8_P-5_P-8_1_1_-3_3
7,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Kyrie,164.0,"[20/3.0, 24/3.0, 27/2.0, 27/4.0, 30/4.0, 36/1....","[P-8, P8, P-8, P5, P4, P5, M-6]","[164.0, 196.0, 218.0, 222.0, 246.0, 288.0, 322...","[2, 2, -2, -3]","[32.0, 22.0, 4.0, 24.0, 42.0, 34.0, 4.0]","[Altus, Bassus, Cantus, Tenor, Tenor, Altus, C...",FUGA,8,False,P-8_P8_P-8_P5_P4_P5_M-6,32.0_22.0_4.0_24.0_42.0_34.0_4.0,2_2_-2_-3,P-8_P8_P-8_P5_P4_P5_M-6_32.0_22.0_4.0_24.0_42....,P-8_P8_P-8_P5_P4_P5_M-6_32.0_22.0_4.0_24.0_42....,P-8_P8_P-8_P5_P4_P5_M-6_2_2_-2_-3
8,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Kyrie,78.0,"[10/4.0, 12/2.0, 14/2.0, 15/3.0]","

In [12]:
filtered = final.loc[final['Number_Entries'] < 20] 
filtered

,Composer,Title,First_Offset,Measures_Beats,Melodic_Entry_Intervals,Offsets,Soggetti,Time_Entry_Intervals,Voices,Presentation_Type,Number_Entries,Flexed_Entries,MINT,TINT,SOG,ALL,ALL_INT,ALL_SOG
0,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,394.0,"[50/2.0, 51/4.0, 53/2.0, 55/2.0, 56/4.0, 59/1....","[P5, P-8, P4, P5, P-8, P8, P-4, P-5]","[394.0, 406.0, 418.0, 434.0, 446.0, 464.0, 486...","[-2, 2, 1, 3]","[12.0, 12.0, 16.0, 12.0, 18.0, 22.0, 4.0, 16.0]","[Altus, Cantus, Bassus, Tenor, Altus, Bassus, ...",FUGA,9,False,P5_P-8_P4_P5_P-8_P8_P-4_P-5,12.0_12.0_16.0_12.0_18.0_22.0_4.0_16.0,-2_2_1_3,P5_P-8_P4_P5_P-8_P8_P-4_P-5_12.0_12.0_16.0_12....,P5_P-8_P4_P5_P-8_P8_P-4_P-5_12.0_12.0_16.0_12....,P5_P-8_P4_P5_P-8_P8_P-4_P-5_-2_2_1_3
1,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,0.0,"[1/1.0, 2/1.0, 6/1.0, 7/1.0, 11/3.0, 14/1.0, 1...","[P-5, P-4, P-5, P8, P5, P-8, P1, P8, P-5, P-4,...","[0.0, 8.0, 40.0, 48.0, 84.0, 104.0, 128.0, 150...","[-3, 3, 2, -2, -3, 2, 2, -2]","[8.0, 32.0, 8.0, 36.0, 20.0, 24.0, 22.0, 16.0,...","[Cantus, Altus, Tenor, Bassus, Altus, Cantus, ...",FUGA,15,True,P-5_P-4_P-5_P8_P5_P-8_P1_P8_P-5_P-4_P-5_P8_P8_...,8.0_32.0_8.0_36.0_20.0_24.0_22.0_16.0_8.0_24.0...,-3_3_2_-2,P-5_P-4_P-5_P8_P5_P-8_P1_P8_P-5_P-4_P-5_P8_P8_...,P-5_P-4_P-5_P8_P5_P-8_P1_P8_P-5_P-4_P-5_P8_P8_...,P-5_P-4_P-5_P8_P5_P-8_P1_P8_P-5_P-4_P-5_P8_P8_...
2,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,298.0,"[38/2.0, 38/4.0, 40/2.0, 42/2.0, 44/1.0, 46/2....","[P-5, P8, P-12, P8, P8, P-11]","[298.0, 302.0, 314.0, 330.0, 344.0, 362.0, 382.0]","[1, 1, -3, 3]","[4.0, 12.0, 16.0, 14.0, 18.0, 20.0]","[Altus, Tenor, Cantus, Bassus, Altus, Cantus, ...",FUGA,7,False,P-5_P8_P-12_P8_P8_P-11,4.0_12.0_16.0_14.0_18.0_20.0,1_1_-3_3,P-5_P8_P-12_P8_P8_P-11_4.0_12.0_16.0_14.0_18.0...,P-5_P8_P-12_P8_P8_P-11_4.0_12.0_16.0_14.0_18.0...,P-5_P8_P-12_P8_P8_P-11_1_1_-3_3
3,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Kyrie,458.0,"[56/2.0, 58/4.0, 61/3.0, 63/2.0]","[P-8, P-5, P12]","[458.0, 478.0, 500.0, 514.0]","[-2, 2, 1, 3]","[20.0, 22.0, 14.0]","[Cantus, Tenor, Bassus, Cantus]",FUGA,4,False,P-8_P-5_P12,20.0_22.0_14.0,-2_2_1_3,P-8_P-5_P12_20.0_22.0_14.0_-2_2_1_3,P-8_P-5_P12_20.0_22.0_14.0,P-8_P-5_P12_-2_2_1_3
4,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Kyrie,510.0,"[62/4.0, 64/2.0]",[P5],"[510.0, 522.0]","[-2, 2, 2, 2]",[12.0],"[Tenor, Altus]",FUGA,2,False,P5,12.0,-2_2_2_2,P5_12.0_-2_2_2_2,P5_12.0,P5_-2_2_2_2
5,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Kyrie,0.0,"[1/1.0, 2/1.0, 6/1.0, 7/1.0]","[P-5, P-4, P-5]","[0.0, 8.0, 40.0, 48.0]","[-3, 3, 2, -2, -3, 2, 2, -2]","[8.0, 32.0, 8.0]","[Cantus, Altus, Tenor, Bassus]",ID,4,True,P-5_P-4_P-5,8.0_32.0_8.0,-3_3_2_-2,P-5_P-4_P-5_8.0_32.0_8.0_-3_3_2_-2,P-5_P-4_P-5_8.0_32.0_8.0,P-5_P-4_P-5_-3_3_2_-2
6,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Kyrie,368.0,"[45/1.0, 45/3.0, 47/3.0, 49/3.0, 51/4.0, 52/2....","[P-5, P8, P-8, P8, P-5, P-8]","[368.0, 372.0, 388.0, 404.0, 422.0, 426.0, 436.0]","[1, 1, -3, 3, 1, 1, -3, 2]","[4.0, 16.0, 16.0, 18.0, 4.0, 10.0]","[Altus, Tenor, Cantus, Bassus, Altus, Tenor, B...",FUGA,7,True,P-5_P8_P-8_P8_P-5_P-8,4.0_16.0_16.0_18.0_4.0_10.0,1_1_-3_3,P-5_P8_P-8_P8_P-5_P-8_4.0_16.0_16.0_18.0_4.0_1...,P-5_P8_P-8_P8_P-5_P-8_4.0_16.0_16.0_18.0_4.0_10.0,P-5_P8_P-8_P8_P-5_P-8_1_1_-3_3
7,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Kyrie,164.0,"[20/3.0, 24/3.0, 27/2.0, 27/4.0, 30/4.0, 36/1....","[P-8, P8, P-8, P5, P4, P5, M-6]","[164.0, 196.0, 218.0, 222.0, 246.0, 288.0, 322...","[2, 2, -2, -3]","[32.0, 22.0, 4.0, 24.0, 42.0, 34.0, 4.0]","[Altus, Bassus, Cantus, Tenor, Tenor, Altus, C...",FUGA,8,False,P-8_P8_P-8_P5_P4_P5_M-6,32.0_22.0_4.0_24.0_42.0_34.0_4.0,2_2_-2_-3,P-8_P8_P-8_P5_P4_P5_M-6_32.0_22.0_4.0_24.0_42....,P-8_P8_P-8_P5_P4_P5_M-6_32.0_22.0_4.0_24.0_42....,P-8_P8_P-8_P5_P4_P5_M-6_2_2_-2_-3
8,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Kyrie,78.0,"[10/4.0, 12/2.0, 14/2.0, 15/3.0]","

### Network Visualization with Louvain Communities

In [13]:

f2 = filtered.groupby('MINT')['Title'].apply(list).reset_index()
f2


,MINT,Title
0,P-4_P-5_P-4,[Missa Veni sponsa Christi: Kyrie]
1,P-5_P-4_P-5,[Missa Veni sponsa Christi: Kyrie]
2,P-5_P-4_P-5_P8_P5_P-8_P1_P8_P-5_P-4_P-5_P8_P8_...,[Veni sponsa Christi]
3,P-5_P8_P-12_P8_P8_P-11,[Veni sponsa Christi]
4,P-5_P8_P-8_P8_P-5_P-8,[Missa Veni sponsa Christi: Kyrie]
5,P-8_P-5_P12,[Missa Veni sponsa Christi: Kyrie]
6,P-8_P8_P-8_P5_P4_P5_M-6,[Missa Veni sponsa Christi: Kyrie]
7,P5,[Missa Veni sponsa Christi: Kyrie]
8,P5_P-8_P4_P5_P-8_P8_P-4_P-5,[Veni sponsa Christi]


In [14]:
pairs = f2.Title.apply(lambda x: list(combinations(x, 2)))
pairs

0    []
1    []
2    []
3    []
4    []
5    []
6    []
7    []
8    []
Name: Title, dtype: object

In [15]:
pairs2 = pairs.explode().dropna()
pairs2

Series([], Name: Title, dtype: object)

In [16]:
unique_pairs = pairs.explode().dropna().unique()
unique_pairs

array([], dtype=object)

In [17]:
pd.Series(unique_pairs).isna().sum()

0

In [18]:
def add_communities(G):
    G = deepcopy(G)
    partition = community_louvain.best_partition(G)
    nx.set_node_attributes(G, partition, "group")
    return G

In [19]:
pyvis_graph = Network(notebook=True, width="1800", height="1400", bgcolor="white", font_color="black")

In [20]:
G = nx.Graph()
G.add_edges_from(unique_pairs)
G = add_communities(G)

In [21]:
pyvis_graph.from_nx(G)

In [22]:
pyvis_graph.show('MINT.html')


#### Below is Development Work

In [ ]:
filtered = output.loc[output['Number_Entries'] < 4] 
filtered

In [ ]:
output = output.loc[output['Presentation_Type'] == "PEN"] 
output

In [ ]:
offset_diffs = [2.0, 1.0, 2.0, 3.0, 5.0, 6.0]
# some_list[start:stop:step]
alt_list = offset_diffs[::2]
alt_list

In [ ]:
# this works with ONE list of offsets

points2 = pd.DataFrame()
split_list = [90.0, 94.0, 102.0, 106.0, 134.0, 146.0, 162.0]

l = len(split_list)  
for r in range(3, l):
    list_combinations = list(combinations(split_list, r))
#             combo_time_ints = []
    for combo in list_combinations:
        combo_time_ints = numpy.diff(combo).tolist()
        combo_array = entry_array[entry_array.index.get_level_values(0).isin(combo)]
        combo_voice_list = combo_array['voice'].to_list()
        combo_patterns = combo_array['pattern']
        unique_combo_patterns = list(set(combo_patterns))
        tone_coordinates =  list(zip(combo, combo_voice_list))
# tone_coordinates.ffill(inplace=True)
        mel_ints = find_entry_int_distance(tone_coordinates, piece)
        hidden_type = classify_by_offset(combo_time_ints)

        meas_beat = det[det.index.get_level_values('Offset').isin(combo)]
        mb2 = meas_beat.reset_index()
        mb2['mb'] = mb2["Measure"].astype(str) + "/" + mb2["Beat"].astype(str)
        meas_beat_list = mb2['mb'].to_list()

        combo_temp = {'First_Offset': combo[0], 
            'Offsets': combo, 
            'Measures_Beats': meas_beat_list,
            'Presentation_Type': hidden_type,
            "Soggetti": unique_combo_patterns,
            'Voices': combo_voice_list, 
            'Time_Entry_Intervals': combo_time_ints, 
            'Melodic_Entry_Intervals': mel_ints}

        if 'PEN' in hidden_type:
            points2 = points2.append(combo_temp, ignore_index=True).sort_values("First_Offset")
#             points2 = points2[points2['Offsets'].apply(len) > 1]
        if 'ID' in hidden_type:
            points2 = points2.append(combo_temp, ignore_index=True).sort_values("First_Offset")
#             points2 = points2[points2['Offsets'].apply(len) > 1]
        
        
# combo_time_ints
# combo_array
# # combo_voice_list
# # combo_patterns
# # unique_combo_patterns
# # tone_coordinates
# # mel_ints
# # combo_temp
points2

In [ ]:
# this finds hidden fugas.  
# try to run each of the first set of results above ('points') through this tool, then append the 
# new results to the full DF, and sort again.  
# mark each long pattern with 'has hidden pattern' boolean?  or ?

sample_list = points["Offsets"][4]

hidden_pts = []
n = len(sample_list)
for item in range(3, n):
    list_combinations = list(combinations(sample_list, item))
    for group in list_combinations:
        group_time_ints = numpy.diff(group).tolist()
        hidden_type = classify_by_offset(group_time_ints)
        if 'PEN' in hidden_type:
            print(group)
            print(group_time_ints)
            print(hidden_type)
            hidden_pts.append(group_time_ints)
        if 'ID' in hidden_type:
            print(group)
            print(group_time_ints)
            print(hidden_type)
            hidden_pts.append(group_time_ints)
        

list_combinations

In [ ]:
def classify_entries_as_presentation_types(piece):
    # Classifier with Functions
    points = pd.DataFrame()
    points2 = pd.DataFrame()
    # new_offset_list = []
    nr = piece.getNoteRest()
    det = piece.detailIndex(nr, offset=True)

    # durations and ngrams of durations
    dur = piece.getDuration(df=nr)
    dur_ng = piece.getNgrams(df=dur, n=4)

    # ngrams of melodic entries
    # for chromatic, use:
    # piece.getMelodicEntries(interval_settings=('c', True, True), n=5)
    mel = piece.getMelodicEntries(n=4)
    mels_stacked = mel.stack().to_frame()
    mels_stacked.rename(columns =  {0:"pattern"}, inplace = True)

    # edit distance, based on side-by-side comparison of melodic ngrams
    # gets flexed and other similar soggetti
    dist = piece.getDistance(mel)
    dist_stack = dist.stack().to_frame()


    # filter distances to threshold.  <2 is good
    filtered_dist_stack = dist_stack[dist_stack[0] < 2]
    filtered_dist = filtered_dist_stack.reset_index()
    filtered_dist.rename(columns =  {'level_0':"source", 'level_1':'match'}, inplace = True)

    # Group the filtered distanced patterns
    full_list_of_matches = filtered_dist.groupby('source')['match'].apply(list).reset_index()

    for matches in full_list_of_matches["match"]:
        related_entry_list = mels_stacked[mels_stacked['pattern'].isin(matches)]
        entry_array = related_entry_list.reset_index(level=1).rename(columns = {'level_1': "voice", 0: "pattern"})
        offset_list = entry_array.index.to_list()
        split_list = list(split_by_threshold(offset_list))
        # here is the list of starting offsets of the original set of entries:  slist
        slist = split_list[0]
        temp = temp_dict_of_details(slist, entry_array, det, matches)

        points = points.append(temp, ignore_index=True)
        points['Presentation_Type'] = points['Time_Entry_Intervals'].apply(classify_by_offset)
        points.drop_duplicates(subset=["First_Offset"], keep='first', inplace = True)
        points = points[points['Offsets'].apply(len) > 1]

        l = len(slist)
        if l > 2:
            for r in range(3, l):
    #             list_combinations = list(combinations(slist, r))
                list_combinations = list(combinations(slist, r))
                for slist in list_combinations:

                    temp = temp_dict_of_details(slist, entry_array, det, matches)

                    temp["Presentation_Type"] = classify_by_offset(temp['Time_Entry_Intervals'])

                    if 'PEN' in temp["Presentation_Type"]:
                        points2 = points2.append(temp, ignore_index=True)#.sort_values("First_Offset")
    #                     points = points.append(combo_temp, ignore_index=True).sort_values("First_Offset")
                        points2 = points2[points2['Offsets'].apply(len) > 1]
                    if 'ID' in temp["Presentation_Type"]:
                        points2 = points2.append(combo_temp, ignore_index=True)#.sort_values("First_Offset")
    #                     points = points.append(combo_temp, ignore_index=True).sort_values("First_Offset")
                points2.sort_values("First_Offset")
                points2.drop_duplicates(subset=["First_Offset"], keep='first', inplace = True)

    points_combined = points.append(points2, ignore_index=True).sort_values("First_Offset").reset_index(drop=True)
    points_combined['Flexed_Entries'] = points_combined["Soggetti"].apply(len) > 1
    points_combined["Number_Entries"] = points["Offsets"].apply(len)     
    return points2


In [ ]:
# This test works

points = pd.DataFrame()
points2 = pd.DataFrame()
# new_offset_list = []
nr = piece.getNoteRest()
det = piece.detailIndex(nr, offset=True)

# durations and ngrams of durations
dur = piece.getDuration(df=nr)
dur_ng = piece.getNgrams(df=dur, n=4)

# ngrams of melodic entries
# for chromatic, use:
# piece.getMelodicEntries(interval_settings=('c', True, True), n=5)
mel = piece.getMelodicEntries(n=4)
mels_stacked = mel.stack().to_frame()
mels_stacked.rename(columns =  {0:"pattern"}, inplace = True)

# edit distance, based on side-by-side comparison of melodic ngrams
# gets flexed and other similar soggetti
dist = piece.getDistance(mel)
dist_stack = dist.stack().to_frame()


# filter distances to threshold.  <2 is good
filtered_dist_stack = dist_stack[dist_stack[0] < 2]
filtered_dist = filtered_dist_stack.reset_index()
filtered_dist.rename(columns =  {'level_0':"source", 'level_1':'match'}, inplace = True)

# Group the filtered distanced patterns
full_list_of_matches = filtered_dist.groupby('source')['match'].apply(list).reset_index()

for matches in full_list_of_matches["match"]:
    related_entry_list = mels_stacked[mels_stacked['pattern'].isin(matches)]
    entry_array = related_entry_list.reset_index(level=1).rename(columns = {'level_1': "voice", 0: "pattern"})
    offset_list = entry_array.index.to_list()
    split_list = list(split_by_threshold(offset_list))
    # here is the list of starting offsets of the original set of entries:  slist
    slist = split_list[0]
    temp = temp_dict_of_details(slist, entry_array, det, matches)

    points = points.append(temp, ignore_index=True)
    points['Presentation_Type'] = points['Time_Entry_Intervals'].apply(classify_by_offset)
    points.drop_duplicates(subset=["First_Offset"], keep='first', inplace = True)
    points = points[points['Offsets'].apply(len) > 1]

    test = [278.0, 286.0, 294.0, 298.0, 306.0, 310.0]

    l = len(test)  
    for item in range(3, l):
        list_combinations = list(combinations(test, item))
        for group in list_combinations:
            group_time_ints = numpy.diff(group).tolist()
            hidden_type = classify_by_offset(group_time_ints)
            for item in group:
    #         print(item)
                array = group[entry_array.index.get_level_values(0).isin(item)]
                short_offset_list = array.index.to_list()
                time_ints = numpy.diff(array.index).tolist()
                voice_list = array['voice'].to_list()
                if 'PEN' in hidden_type:
                    print(group)
                    print(group_time_ints)
                    print(hidden_type)
                    hidden_pts.append(group_time_ints)
                if 'ID' in hidden_type:
                    print(group)
                    print(group_time_ints)
                    print(hidden_type)
                    hidden_pts.append(group_time_ints)
# len(split_list[0])           

In [ ]:
#  This shows how the classifier works:

if len(set(offset_diffs)) == 1 and len(offset_diffs) > 1:
    print('This is a PEN')
    # elif (len(offset_difference_list) %2 != 0) and (len(set(alt_list)) == 1):
elif (len(offset_diffs) % 2 != 0) and (len(set(alt_list)) == 1) and (len(offset_diffs) >= 3):
    print('This is an ID')
elif len(offset_diffs) >= 1:
    print('This is a FUGA')

In [ ]:
# This shows how combinations works for a given set of time intervals
offset_diffs = [12.0, 32.0, 12.0, 4.0]
l = len(offset_diffs)
# print(l)
if l > 2:
    for r in range(3, l):
        print(r)
        list_combinations = list(combinations(offset_diffs, r))
#         for slist in list_combinations:
        print(list_combinations)

In [ ]:
slist = [278.0, 286.0, 294.0, 298.0, 306.0, 310.0]
l = len(slist)
# for r in range(3, 6):
list_combinations = list(combinations(slist, 4))
#     for tiny_list in list_combinations:

In [ ]:
print(list_combinations)

In [ ]:
list_offsets = [294.0, 298.0, 306.0, 310.0]

In [ ]:
offset_diffs = [4, 5, 6]

In [ ]:
alt_list = offset_diffs[::2]

if len(set(offset_diffs)) == 1 and len(offset_diffs) > 1:
    print('This is a PEN')
    # elif (len(offset_difference_list) %2 != 0) and (len(set(alt_list)) == 1):
elif (len(offset_diffs) % 2 != 0) and (len(set(alt_list)) == 1) and (len(offset_diffs) >= 3):
    print('This is an ID')
elif len(offset_diffs) >= 1:
    print('This is a FUGA')